In [17]:
# 1. وارد کردن کتابخانه‌ها
import pandas as pd
import numpy as np
import torch
from torch.nn import Linear, Dropout, BatchNorm1d, ReLU
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

# 2. دانلود و بارگذاری داده‌ها
def download_data():
    url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
    columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 
               'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
    data = pd.read_csv(url, header=None, names=columns)
    return data

data = download_data()

# 3. پیش‌پردازش داده‌ها
columns_to_check = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in columns_to_check:
    data[col] = data[col].replace(0, np.nan)
    data[col].fillna(data[col].mean(), inplace=True)

X = data.drop(columns=['Outcome']).values
y = data['Outcome'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

# تبدیل داده‌ها به Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# تقسیم داده‌ها به آموزشی و آزمایشی
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# 4. تعریف مدل NN
class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = Linear(input_dim, hidden_dim)
        self.bn1 = BatchNorm1d(hidden_dim)
        self.fc2 = Linear(hidden_dim, hidden_dim)
        self.bn2 = BatchNorm1d(hidden_dim)
        self.fc3 = Linear(hidden_dim, output_dim)
        self.dropout = Dropout(0.3)
        self.relu = ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# تنظیمات اولیه مدل
input_dim = X.shape[1]
hidden_dim = 256
output_dim = 2

model = NeuralNetwork(input_dim, hidden_dim, output_dim)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 5. تعریف بهینه‌ساز و تابع هزینه
optimizer = Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# 6. آموزش مدل
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 7. ارزیابی مدل
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output = model(X_batch)
            _, preds = torch.max(output, 1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    acc = correct / total
    return acc, all_labels, all_preds

# حلقه آموزش
num_epochs = 100
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, optimizer, criterion, device)
    acc, _, _ = evaluate_model(model, test_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {acc:.4f}")

# گزارش نهایی
acc, y_true, y_pred = evaluate_model(model, test_loader, device)
print("\nگزارش عملکرد:")
print(classification_report(y_true, y_pred))


Epoch 1/100, Loss: 0.5098, Accuracy: 0.7078
Epoch 2/100, Loss: 0.4657, Accuracy: 0.7143
Epoch 3/100, Loss: 0.4338, Accuracy: 0.7468
Epoch 4/100, Loss: 0.4175, Accuracy: 0.7403
Epoch 5/100, Loss: 0.4319, Accuracy: 0.7468
Epoch 6/100, Loss: 0.4211, Accuracy: 0.7273
Epoch 7/100, Loss: 0.4855, Accuracy: 0.7403
Epoch 8/100, Loss: 0.4407, Accuracy: 0.7468
Epoch 9/100, Loss: 0.4416, Accuracy: 0.7468
Epoch 10/100, Loss: 0.3955, Accuracy: 0.7662
Epoch 11/100, Loss: 0.4311, Accuracy: 0.7468
Epoch 12/100, Loss: 0.4183, Accuracy: 0.7662
Epoch 13/100, Loss: 0.4369, Accuracy: 0.7532
Epoch 14/100, Loss: 0.4040, Accuracy: 0.7338
Epoch 15/100, Loss: 0.3887, Accuracy: 0.7597
Epoch 16/100, Loss: 0.3772, Accuracy: 0.7532
Epoch 17/100, Loss: 0.3760, Accuracy: 0.7403
Epoch 18/100, Loss: 0.3782, Accuracy: 0.7208
Epoch 19/100, Loss: 0.4112, Accuracy: 0.7468
Epoch 20/100, Loss: 0.3812, Accuracy: 0.7143
Epoch 21/100, Loss: 0.3686, Accuracy: 0.7338
Epoch 22/100, Loss: 0.3992, Accuracy: 0.7662
Epoch 23/100, Loss: